# 라이브러리

In [1]:
from wav2vec2_vib import Model
import torch
from torchinfo import summary

# 모델 불러오기

In [2]:
device = "cpu"
# device = "cuda" if torch.cuda.is_available() else "cpu"
model = Model(
    device=device,
    ssl_cpkt_path='xlsr2_300m.pt', # https://github.com/facebookresearch/fairseq/blob/main/examples/wav2vec/xlsr/README.md#:~:text=XLS%2DR%20300M-,download,-XLS%2DR%201B
).to(device)
model.load_state_dict(torch.load('vib_conf-5_gelu_2s_may27_epoch6.pth', map_location=device))

/home/woonj/anaconda3/envs/grad_cam/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


<All keys matched successfully>

## 모델 구조

In [3]:
# summary(model, input_size=(8, 64600))
summary(model)
# 8 : batch size
# 64600 : 4 seconds

Layer (type:depth-idx)                                            Param #
Model                                                             --
├─SSLModel: 1-1                                                   --
│    └─Wav2Vec2Model: 2-1                                         1,024
│    │    └─ConvFeatureExtractionModel: 3-1                       4,210,176
│    │    └─Linear: 3-2                                           525,312
│    │    └─Dropout: 3-3                                          --
│    │    └─Dropout: 3-4                                          --
│    │    └─GumbelVectorQuantizer: 3-5                            574,080
│    │    └─Linear: 3-6                                           590,592
│    │    └─TransformerEncoder: 3-7                               310,701,184
│    │    └─LayerNorm: 3-8                                        1,024
│    │    └─Linear: 3-9                                           787,200
├─Linear: 1-2                                           

In [4]:
model

Model(
  (ssl_model): SSLModel(
    (model): Wav2Vec2Model(
      (feature_extractor): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
            (0): Conv1d(512, 512, kernel_size=

## 모델 레이어 선택

In [10]:
#ConvFeatureExtractionModel의 마지막 합성곱 레이어
model.ssl_model.model.feature_extractor.conv_layers[-1]

Sequential(
  (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
  (1): Dropout(p=0.0, inplace=False)
  (2): Sequential(
    (0): TransposeLast()
    (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): TransposeLast()
  )
  (3): GELU(approximate='none')
)

## 예측 확률값 받아오기

In [ ]:
from torch import Tensor
import numpy as np


tensor_input = torch.from_numpy(audio)
tensor_input = tensor_input.unsqueeze(0)


with torch.no_grad(): # 추론할 때는 가중치를 업데이트할 필요가 없기 때문에 그라디언트 계산 안해도 됨

    output = model(tensor_input.to(device))


In [ ]:
# softmax to convert output to probabilities


probabilities = torch.nn.functional.softmax(output, dim=1) * 100

print(probabilities[0][0]) # spoof probability
print(probabilities[0][1]) # bonafide probability